In [1]:
myth_entities = {
    "Zeus": ["Zeus"],
    "Apollo": ["Apollo", "Phoebus"],
    "Athena": ["Athena", "Pallas"],
    "Artemis": ["Artemis"],
    "Hades": ["Hades", "Plouton"],
    "Poseidon": ["Poseidon"],
    "Demeter": ["Demeter"],
    "Hera": ["Hera"],
    "Hermes": ["Hermes"],
    "Ares": ["Ares"],
    "Dionysus": ["Dionysus", "Bacchus"],
    "Hephaestus": ["Hephaestus", "Vulcan"],
    # You can add heroes too, like "Heracles": ["Heracles", "Hercules"]
}


In [2]:
import spacy
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from time import sleep
import csv

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2_000_000
geolocator = Nominatim(user_agent="myth_mapper")

# Scrape Apollodorus
url = "https://www.theoi.com/Text/Apollodorus1.html"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
apollodorus_text = soup.get_text(separator=" ")

# Load Homer texts
odyssey = requests.get("https://www.gutenberg.org/files/1727/1727-0.txt").text
iliad = requests.get("https://www.gutenberg.org/files/6130/6130-0.txt").text

sources = {
    "Apollodorus": apollodorus_text,
    "Odyssey": odyssey,
    "Iliad": iliad
}

myth_entities = {
    "Zeus": ["Zeus"],
    "Apollo": ["Apollo", "Phoebus"],
    "Athena": ["Athena", "Pallas"],
    "Artemis": ["Artemis"],
    "Hades": ["Hades", "Plouton"],
    "Poseidon": ["Poseidon"],
    "Demeter": ["Demeter"],
    "Hera": ["Hera"],
    "Hermes": ["Hermes"],
    "Ares": ["Ares"],
    "Dionysus": ["Dionysus", "Bacchus"],
    "Hephaestus": ["Hephaestus", "Vulcan"]
}

output_data = []

for source_name, raw_text in sources.items():
    doc = nlp(raw_text)
    ents = [(ent.text, ent.start_char, ent.label_) for ent in doc.ents if ent.label_ in ("GPE", "LOC")]

    for place, pos, _ in ents:
        # Check nearby tokens for mythological names
        context_window = raw_text[max(0, pos-200):pos+200].lower()
        associated_god = None
        for god, aliases in myth_entities.items():
            if any(alias.lower() in context_window for alias in aliases):
                associated_god = god
                break

        # Skip if we already have this place-god combo
        if not associated_god:
            continue

        # Try geocoding the place
        try:
            loc = geolocator.geocode(place)
            if loc:
                output_data.append({
                    "Place": place,
                    "Latitude": loc.latitude,
                    "Longitude": loc.longitude,
                    "God": associated_god,
                    "Source": source_name
                })
                sleep(1)
        except:
            continue


In [3]:
with open("mythology_locations.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["Place", "Latitude", "Longitude", "God", "Source"])
    writer.writeheader()
    for row in output_data:
        writer.writerow(row)
